# Wyndor Glass Co. 使用 Gurobi 的線性規劃問題

在這個例子中，我們將使用 Python 中的 Gurobi 來建模和解決 Wyndor Glass Co. 的線性規劃問題。目標是在考慮三個工廠的生產能力的情況下，最大化生產兩種產品的總利潤。

**問題描述：**

- **決策變數：**
  - $x_1$：每週生產的產品 1 的批次數。
  - $x_2$：每週生產的產品 2 的批次數。

- **目標函數：** 
  $$
  \text{Maximize } Z = 3x_1 + 5x_2
  $$
  其中 $Z$ 是每週的總利潤（以千美元計）。

- **約束條件：**
  - 工廠 1 產能：$x_1 \leq 4$
  - 工廠 2 產能：$2x_2 \leq 12$
  - 工廠 3 產能：$3x_1 + 2x_2 \leq 18$
  - 非負性：$x_1 \geq 0$, $x_2 \geq 0$

## 步驟 1：導入 Gurobi 並設置環境

Gurobi 是一個強大的優化求解器，可以處理線性規劃、整數規劃和其他類型的數學優化問題。我們導入 `gurobipy` 模組以訪問其函數和類。


In [8]:
import gurobipy as gp
from gurobipy import GRB

## 步驟 2：創建新模型

我們創建一個新的模型對象，在其中添加我們的決策變數、目標函數和約束條件。為模型命名有助於識別它，特別是在處理多個模型時。


In [9]:
# 創建新模型
model = gp.Model('Wyndor_Glass_Co')

## 步驟 3：定義決策變數

決策變數代表我們要確定的數量。我們將 \( x_1 \) 和 \( x_2 \) 定義為連續變數，下界為 0（因為我們不能生產負數的產品批次）。`vtype` 參數設置為 `GRB.CONTINUOUS`，表示變數可以取任何非負實數值。

In [10]:
# 添加決策變數
x1 = model.addVar(name='x1', vtype=GRB.CONTINUOUS, lb=0)
x2 = model.addVar(name='x2', vtype=GRB.CONTINUOUS, lb=0)

## 步驟 4：設置目標函數

目標函數代表優化問題的目標。在這個案例中，我們要最大化總利潤，計算為 \( 3x_1 + 5x_2 \)。`setObjective` 方法設置這個函數，我們指定 `GRB.MAXIMIZE` 來表示我們正在最大化目標。

In [11]:
# 設置目標函數
model.setObjective(3 * x1 + 5 * x2, GRB.MAXIMIZE)

## 步驟 5：添加約束條件

約束條件定義了解決方案必須滿足的限制或要求。我們使用 `addConstr` 方法將每個約束條件添加到模型中。

- **工廠 1 產能約束**：\( x_1 \leq 4 \)
  - 限制產品 1 的生產每週最多 4 批。
- **工廠 2 產能約束**：\( 2x_2 \leq 12 \)
  - 因為產品 2 的每批使用工廠 2 的 2 單位產能，總產能為 12 單位。
- **工廠 3 產能約束**：\( 3x_1 + 2x_2 \leq 18 \)
  - 代表工廠 3 對兩種產品的綜合產能限制。

每個約束條件都有一個名稱，便於識別。

In [12]:
# 添加約束條件
model.addConstr(x1 <= 4, name='Plant1_Capacity')
model.addConstr(2 * x2 <= 12, name='Plant2_Capacity')
model.addConstr(3 * x1 + 2 * x2 <= 18, name='Plant3_Capacity')

<gurobi.Constr *Awaiting Model Update*>

## 步驟 6：優化模型

模型完全定義後，我們調用 `optimize` 方法來解決優化問題。Gurobi 使用先進的算法來有效地找到最優解。

In [13]:
# 優化模型
model.optimize()

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[arm] - Darwin 24.1.0 24B5046f)

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 3 rows, 2 columns and 4 nonzeros
Model fingerprint: 0xc030cf3a
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [3e+00, 5e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+00, 2e+01]
Presolve removed 2 rows and 0 columns
Presolve time: 0.00s
Presolved: 1 rows, 2 columns, 2 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.5000000e+01   1.500000e+00   0.000000e+00      0s
       1    3.6000000e+01   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.00 seconds (0.00 work units)
Optimal objective  3.600000000e+01


In [14]:
# 檢查模型是否有最優解
if model.status == GRB.OPTIMAL:
    print("找到最優解：")
    print(f"  x1（每週產品 1 的批次數）= {x1.x}")
    print(f"  x2（每週產品 2 的批次數）= {x2.x}")
    print(f"  最大利潤（以千美元計）= {model.objVal}")
else:
    print("未找到最優解。")

找到最優解：
  x1（每週產品 1 的批次數）= 2.0
  x2（每週產品 2 的批次數）= 6.0
  最大利潤（以千美元計）= 36.0


## 結果解釋

最優解提供了在滿足所有生產能力約束的同時，能最大化利潤的每種產品的批次數。最大利潤值表示每週的總利潤，以千美元計。